In [3]:
%load_ext autoreload
%autoreload 2
import os
import sys
from IPython.display import IFrame

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import fmm
from shapely.geometry import LineString
import swifter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
network = RoadNetwork("Porto, Portugal", network_type="drive", retain_all=True, truncate_by_edge=True)

In [ ]:
import folium
gdf_nodes, gdf_edges = ox.graph_to_gdfs(network.G)
bounds = gdf_nodes.geometry.total_bounds
print(bounds)
m = folium.Map(
    location=[41.1372482, -8.689151],
    zoom_start=11,
)

folium.Rectangle(bounds=[(bounds[1], bounds[0]),(bounds[3], bounds[2])], color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(m)
m

In [ ]:
from shapely.geometry import box

bbox = box(*bounds)
poly_gdf = gpd.GeoDataFrame([1], geometry=[bbox], crs="EPSG:4326")
poly_gdf

In [ ]:
from shapely import wkt
df = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")
df['POLYLINE'] = df['POLYLINE'].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="POLYLINE")

In [ ]:
gdf_clipped = gdf.clip(poly_gdf, keep_geom_type=True)

In [6]:
network.save(path="../osm_data/porto")


/home/paul/repos/Road-Network-Embedding-Generator/generator/road_network.py:73: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(path + "/nodes.shp", encoding="utf-8")


In [7]:
network.fmm_trajectorie_mapping()

gps file : ../datasets/trajectories/Porto/mapped_id_poly_clipped_timestamp.csv
id column : id
geom column : POLYLINE
timestamp column : timestamp
x column : x
y column : y
GPS point : false
[2022-05-19 12:22:32.146] [info] [network.cpp:72] Read network from file ../osm_data/porto/edges.shp

Result file : ../datasets/trajectories/Porto/road-segment-mapping.txt
Output fields: opath spdist cpath mgeom duration speed 
Status: success
Time takes 16384.6 seconds
Total points 62456473 matched 52263949
Map match speed 3189.82 points/s 

[2022-05-19 12:22:32.225] [info] [network.cpp:170] Number of edges 11331 nodes 5358
[2022-05-19 12:22:32.225] [info] [network.cpp:171] Field index: id 17 source 0 target 1
[2022-05-19 12:22:32.231] [info] [network.cpp:174] Read network done.
[2022-05-19 12:22:32.231] [info] [network_graph.cpp:17] Construct graph from network edges start
[2022-05-19 12:22:32.233] [info] [network_graph.cpp:30] Graph nodes 5358 edges 11331
[2022-05-19 12:22:32.233] [info] [network

In [ ]:
G = network.G
gdf_nodes, gdf_edges = ox.graph_to_gdfs(
        G,
        nodes=True, edges=True,
        node_geometry=True,
        fill_edge_geometry=True)

gdf_edges.describe(include='all')

In [ ]:
m1 = ox.plot_graph_folium(G, popup_attribute="name", weight=2, color="#FFFFFF", tiles="cartodbdark_matter")
filepath = "graph.html"
m1.save(filepath)
#IFrame(filepath, width=600, height=500)

In [ ]:
# load porto trajectory dataset
td = pd.read_csv("../datasets/trajectories/Porto/train.csv")

In [ ]:
td = td[td["MISSING_DATA"]==False]
sample = np.matrix(td.loc[0, "POLYLINE"]).reshape(-1, 2)
gtd = gpd.GeoDataFrame(data=sample, index=np.arange(1, len(sample)+1), columns=["LONG", "LAT"], crs="EPSG:4326", geometry=gpd.points_from_xy(sample[:, 0], sample[:, 1]))
print(gtd.to_wkt())
#network.map_trajectorie(coordinates=gtd)

In [8]:
df = td = pd.read_csv("../datasets/trajectories/Porto/road-segment-mapping.csv", sep=";")

In [13]:
#df.dropna(inplace=True)
df.head()

,id,opath,spdist,cpath,mgeom,duration,speed
0,50002,"3333,3321,3321,3276,1944,1944,1967,1953,8329,1...","0.000199229,0.000891484,0.00122959,0.00183949,...","3333,3321,3276,5080,9347,1944,1956,1972,2157,1...","LINESTRING(-8.61323453196 41.1666160384,-8.613...","15,15,15,15,15,15,15,15,15,15,15,15,15,15,15","1.32819566794e-05,5.94322398265e-05,8.19724834..."
1,33335,"7157,8962,8962,1025,1025,1025,1025,2203,4764,2...","0.000651735,0.0019678,0.00192088,0.000329327,3...","7157,902,8962,1025,1015,2203,4759,4764,4770,20...","LINESTRING(-8.5985652 41.1483815,-8.5985453 41...","15,15,15,15,15,15,15,15,15,15,15,15","4.34489891046e-05,0.000131186512154,0.00012805..."
3,1,"527,527,1104,1104,525,525,248,248,245,3475,524...","0.000146027,0.00446739,0.00227141,0.00188481,0...","527,1104,525,248,3492,8674,3491,245,3475,5242,...","LINESTRING(-8.61862328377 41.1414555366,-8.618...","15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,1...","9.73512544027e-06,0.000297826229,0.00015142704..."
4,83335,"813,8474,8975,8975,8975,982,1095,1095,3432,808...","0.00034158,0.000836386,0.00120258,0.00044536,0...","813,8474,7886,7859,8975,982,1095,3432,8081,785...","LINESTRING(-8.61086059556 41.1456506984,-8.610...","15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,1...","2.27720037272e-05,5.57590548845e-05,8.01722659..."
6,50004,"6565,10292,10292,10576,1008,993,993,995,995,99...","7.6059e-05,9.13997e-05,0.000740901,0.00102824,...","6565,10292,10576,1008,993,995","LINESTRING(-8.59609195195 41.1417507653,-8.596...","15,15,15,15,15,15,15,15,15,15,15","5.07059754091e-06,6.09331097111e-06,4.93933875..."


In [15]:

x = 5e-5

x

5e-05